# Ordinary Differential Equations (Euler's & Runge-Kutta Method)

Today we'll do use the tools we've built up to solve a physics problem: use python to compute the motion of a projectile in the presence of drag! One method is a basic algorithm and the other is more complex.

## Initial Learning Objectives
* Practice with algorithms and functions
* Learn how to discretize a differential equation
* Do a few simple problems using the Euler method
* Learn about numerical discretization error
* Relate a mathematical solution to its physical interpretation

## Additional Learning Objectives
* Better algorithms > faster computers
* Write a slightly more complex algorithm
* Practice using external libraries

##  Nuclear decay

Let's first calculate and plot the analytic solution of a nuclear decay problem so we can compare our numerical solutions we'll enact later.

Unstable nuclei (like ${}^{235}$U) can undergo radioactive decay and become two smaller nuclei along with other decay products. As a result of the uncertainity principle, we cannot know precisely when a particular nucleus will decay. But, we can know the quantum mechanical probability that it will have decayed after some time.

If we consider a very large number of nuclei, as opposed to, say, a single ${}^{235}$U nucleus, we can examine what the probability means for a physical system. If we know the probability that a single nucleus will decay after some period of time, then given a large number of nuclei, we can determine the fraction that remain after that period of time. 

We know that the number of nuclei that will decay after some time is proportional to the number that were there to begin with and as a result, the rate of change of the number of nuclei is proportional to the number of nuclei. That is, $\frac{dN(t)}{dt}$ is proportional to $N(t)$, where $N(t)$ is the number of nuclei present at time t. Thus, the evolution of $N(t)$ is given by

$\quad \frac{dN(t)}{dt} = -\frac{1}{\tau} N(t)$

where the minus sign reflect the fact that the population decreases in time. The constant $\tau$ is the mean lifetime (not the half–life!) of the unstable nucleus.

You may recall that the resulting decay is _exponential_, that is, given an initial number of nuclei $N(t=0)\equiv N_0$,

$\quad N(t) = N_0 \exp\left(-\frac{t}{\tau}\right)$

This can be obtained via separation of variables.  We rewrite the differential equation as

$\quad \frac{dN}{N} = -\frac{dt}{\tau}$

integrate to find

$\mathbf{\quad \ln(N) = -\frac{t}{\tau} + C}$

and solve for $N(t)$ obtaining the relation above.  The _half-life_ is related to $\tau$: $t_h = \tau \ln 2$.

## Exercise

Let's plot this result. You're given initial values of $N_0 = 100$ and a mean lifetime $\tau=0.7$gy. 
Now make an array of times and calculate the number of nuclei at each time point. A time range from $t=0$gy to $t=4$gy is suitable for this example.

Note: gy = gigayear, or $10^{9}$ years.


In [ ]:
import numpy as np
import pylab as p

tmin = 0.0
tmax = 4.0
nts = 100

N0 = 100.0  # Note:  I'm treating this as a floating point number even though the number of nuclei should probably
            # be an integer.  We've assumed a large-ish number, so that any fraction of the initial is a whole number.
tau = 0.7


t_array = # Produce an array that runs from 0 to 4 (in units of gy) with 100-ish points
N_array = # Compute the number of nuclei. This can be done directly with array math

p.plot(t_array,N_array,'-')
p.xlabel("Time [gy]")
p.ylabel("Number of nuclei")
p.show()


Nuclear decay with a numerical differential equation solver -- Euler's method
------------------------------
We can think about this process another way.  At any given time $t$, we know how to determine the
number of nuclei at the next time:

$\quad\frac{dN(t)}{dt} = -\frac{1}{\tau}N(t) \quad\rightarrow\quad N(t+dt) = N(t) - \frac{dt}{\tau} N(t)$

It is also useful to re-write this using an index notation -- instead of using the time $t$ as a continuous argument, we recall that we typically have an array of times (i.e. it has been _discretized_). We can index the times with $t_i$, and can also write the discretized differential equation as

$N(t_{i+1}) = N(t_i) - \frac{\Delta t}{\tau} N(t_i)$

or even shorter

$N_{i+1} = N_i - \frac{\Delta t}{\tau} N_i$


Let's try to imagine this as an algorithm:
1. We know the initial value of N(t=0) -- let's index that by it=0
2. Starting from it=0, determine N[it+1] from N[it] through the relation above
3. Stop once we reach the maximum number of time points-1

And, as pseudocode:

    t_array := a time array that contains the time values we care about. it runs from tmin to tmax and 
               has N points in it.
               
    N_array := an array of the same size as t_array where we'll store the number of nuclei
    
    N[0] = N0  # set the initial value
    for each i from 0 to N-1:
        the next value of N (N[i+1]) is determined by the current one N[i] + dt * the slope
        the slope is the RHS of the differential equation, namely -(1/tau) N

The code below implements this approach: walk through it, making sense of each line as you go.

In [ ]:
import numpy as np
import pylab as p

tau = 0.7        # mean lifetime
N0 = 100.0 # initial number of nuclei

tmin = 0.0
tmax = 4.0

nts=20

# Create an array to hold the times
t_array = np.linspace(tmin, tmax, nts, endpoint=False)  
# NOTE ENDPOINT HERE: can you figure out what it does? 

dt = t_array[1] - t_array[0]  
# This is an easy way to define dt that is not error prone
# Should match (tmax-tmin)/nts if you're careful about "endpoint"
        
print("t_array:",t_array)  # See what it looks like

# Empty array to hold the number of nuclei
N_array          = np.zeros( nts )

# THIS IS THE ACTUAL ALGORITHM PART

# Set the initial condition
N_array[0] = N0

#fill the rest of the solution, step by step
for it in range(0, len(t_array)-1 ): # Note the -1: what happens if we don't have it?
    
    N_array[it+1] = N_array[it] + (-dt/tau) * N_array[it]
    
    # Some output so we can watch!
    print('at time t = {:.3f}, N = {:.3f}'.format(t_array[it+1],N_array[it+1]))

p.plot(t_array, N_array, 'x', label="Euler Method with nts="+str(nts))

p.legend()
p.axhline(0.0, color='k', linestyle='--')
p.show()

Exercise
----------------
* Make sure you understand how the above code works. 
* Run this code using various values of `nts`, and compare your answers to the exact answer (you can print this from the cell above).

Note that your answers improve as the resolution is increased. (We say that the resolution increases as the number of timesteps `nts` increases for a given time interval `tmax - tmin`. Equivalently, the resolution increases as the timestep `dt` decreases.)

* Add to the above code: **Plot the analytic solution on the same plot as the points from the differential equation solver.**

## Discretization error

As you should have noticed above, when `nts` isn't very big, the numerical result doesn't quite match the analytic result.  This is because of _discretization error_.  It arises because we made the following approximation:

$\quad \frac{dN(t)}{dt} \approx \frac{N(t+\Delta t) - N(\Delta t)}{\Delta t}$

Let's do a _Taylor expansion_ of $N(t+\Delta t)$ around $\Delta t=0$.

$\quad N(t + \Delta t) = N(t) + \left(\frac{dN(t)}{dt}\right) \Delta t + \frac{1}{2} \left(\frac{d^2 N(t)}{dt^2}\right) \Delta t^2 +\ ...$

Or, writing this with an index notation,

$\quad N_{i+1} = N_i + \left(\frac{dN(t)}{dt}\right)\bigg\vert_{t_i} \Delta t + \frac{1}{2} \left(\frac{d^2 N(t)}{dt^2}\right)\bigg\vert_{t_i} \Delta t^2 +\ ...$

Putting in the expression we arrived at above,

$\quad N_{i+1} = \left[ N_i - \frac{1}{\tau} N_i \Delta t \right] + \frac{1}{2} \left(\frac{d^2 N(t)}{dt^2}\right)\bigg\vert_{t_i} \Delta t^2 +\ ...$

Notice that the thing we calculated above (the term in square brackets) isn't _quite_ the right answer, it misses the $2^\mathrm{nd}$ order and above terms.  This leads to some error, which is known as _discretization_ error. Let's find out how bad this is.

Exercise
-------------

* Using the code above, compare the Euler's method result to the exact analytic result at $t=4$. 
* Run the code using `nts=20,40,80,160,320`.  
* Make a plot of the __relative__ error `(numerical-analytic)/analytic)` at the final time point for those values of `nts` versus `dt`. (you may wish to make an array or list to store the numbers in).  
* How does the error change when you double the resolution?

In [ ]:
# You can copy the code above and work in this cell if needed

If all goes well, you should find that the error is linearly proportional to `dt`. There are two reasons for this:
1. We have neglected terms of order $\Delta t^2$ and up (because we have more than a single term the error isn't precisely linear).
2. Each step accumulates this error, and we have to add all of them up. We have `nts` steps, and `nts` $\propto$ `1/dt`.

The product of these two leads to an error at the endpoint that is proportional to `dt`.



## Euler's method, in general (reference)


Euler’s method can be applied to any system of first order ordinary differential equations (ODE’s). Let $u(t)$ denote a __vector__ of unknowns, and $F(u, t)$ denote a vector of right–hand sides. Then a system of ODE’s can be written as:

$\quad \frac{d{\bf u}(t)}{dt} = {\bf F}({\bf u},t)$

The dependent variables are the components of ${\bf u}$, and the independent variable is $t$.

For example, let's take the system for the variables $x(t), y(t), z(t)$:

$$
\begin{aligned}
dx/dt &= x y - z + t \\
dy/dt &= 2 z + x \\
dz/dt &= t^2 - y z
\end{aligned}
$$

This can be written in the above notation using

$$
{\bf u}(t) = 
\left(\begin{array}{c} 
x(t) \\
y(t) \\
z(t)
\end{array}
\right)
$$
and
$$
{\bf F}({\bf u},t) = 
\left(\begin{array}{c} 
x y - z + t \\
2 z + x \\
t^2 - y z
\end{array}
\right)
$$

Note that in this case, the RHS of the differential equation depends explicitly on $t$, which wasn't the case for nuclear decay.

Discretizing these equations, and applying the Euler's method, we then find

$$
u^j_{i+1} = u^j_i + F^j(u_i,t) \Delta t
$$

where $j$ is the $j^\mathrm{th}$ component of ${\bf u}$, and $i$ indexes the time variable (just as above).  Expanding this back out into $(x,y,z)$, we find:

$$
\begin{aligned}
x_{i+1} &= x_i + \left( x_i y_i - z_i + t_i\right) \Delta t \\
y_{i+1} &= y_i + \left( 2 z_i + x_i\right) \Delta t \\
z_{i+1} &= z_i + \left(t_i^2 - y_i z_i\right) \Delta t
\end{aligned}
$$

The process of converting the continuous ODE's into discrete equations is  _discretization_.

## Practice Problems (On Your Own Time)

### Free Fall equations (without drag)

Consider an object freely falling along the vertical $y$ axis under the influence of gravity. Newton’s second law tells us that the time rate of change of velocity is $dv/dt = -g$, where $g$ is the acceleration due to gravity. The velocity is defined as the time rate of change of position, $v = dy/dt$. Thus, the motion of the object is governed by the ODE’s

$$
\begin{aligned}
\frac{dy}{dt} &= v \\
\frac{dv}{dt} &= -g
\end{aligned}
$$

We can combine these two into a single second order differential equation:
$$
\frac{d^2 y}{dt^2} = -g
$$
but for numerical purposes, the first order form is easier to use.

### With drag

The following image from the lab of Prof. [Lydia Bourouiba](http://lbourouiba.mit.edu) (MIT) was created by leaving a camera running during a cough, so that it shows the $x$ vs. $y$ trajectories of many cough droplets at once

![image](http://lbourouiba.mit.edu/sites/default/files/styles/large/public/coughdroplets_0.jpg?itok=HzamlEQl)

Note that these shapes are not the familiar, symmetric parabolas we are used to seeing for kinematic motion problems. We will model this behavior using a  _Stokes' drag force_ that is proportional to the speed of the droplet, an approximation that is valid only if the airflow isn't too turbulent. 

$$
\begin{aligned}
\frac{dy}{dt} &= v \\
\frac{dv}{dt} &= -g - \beta v
\end{aligned}
$$

The pseudocode for the algorithm looks like:

    set the initial values of y (y[0]) and v (v[0])
    
    for each time point i, use the information from that point to determine the position and 
    velocity at i+1:
    
        determine v[i+1] from v[i] and the derivative of velocity (acceleration)
        determine y[i+1] from y[i] and the derivative of position (velocity, i.e. v[i])

## Excercises (On Your Own Time)


The first 5 are with no drag, i.e. with $\beta=0$ and you can check your work against the kinematic equations. In #6 you'll run some of the same code again _with_ drag.

1) Use Euler’s method to discretize the equations of motion for the position and velocity in the cell above (i.e. physically write out the discretization).

2) Write a python code that uses Euler’s method to solve for the motion of an object in free fall.  Note that you should solve them *at the same time* -- you can't generally first solve for $v$ and then for $y$. Choose the initial conditions $y_0 = 12.0m$ and $v_0 = 35.0 m/s$. Let $g = 9.8 m/s^2$.

3) Use your code to determine the height for times $0 \leq t \leq 8 s$, with evenly-spaced values of $t$. Plot a graph of y versus t. (Make sure your graph is properly labeled.)

4) Solve for the height of the object analytically, and (for the given initial conditions) find the exact height at $t = 8 s$.

5) Have your code compute the relative error -- |approximate-true|/|true|  -- in the height at $t = 8 s$. Do this for at least four different _resolutions_ (aka number of points). Show that the error is approximately cut in half when the resolution is doubled.

6) One of the big advantages of having a computer solve differential equations is that we can solve things that cannot be done easily analytically. To demonstrate this, add a term in the acceleration representing air friction.  Choose a resolution from part 5. that you're happy with. Make a plot of several trajectories with varying values of $\beta$, including $\beta= 0$, and also plot the analytic result for $\beta=0$ (only). (Note: definitely copy your own code from earlier in the problem: the aim is to have all the curves on the same axes.)

### For (2):

Your code for item 2, implement as a function (name suggestion: `freefall`) so that it takes

* **Input:** `t, y0, v0, g, beta` (in that order, where `t` is a numpy array of times at which to find $y(t)$ and the other 4 values are floating point numbers)
* **Output:** numpy array `y` (together with `t`, this will make $y(t)$, so those two arrays therefore need to have the same length)
* the function call for item 2 would therefore look like `y = freefall(t, 12.0, 35.0, 9.8, 0)` assuming that the array `t` had been previously defined 
* one of the automatic-checks done by nbgrader will be item 4, but for a different time than $t=8$. You can prepare for this by checking that your code works for a few values of $t$.

### As you work:
* Make sure you properly label all axes/titles!
* Do these plots make sense? Why? 
* How do you justify your choices for ranges on the axes?


# Other ODE solvers: Runge-Kutta

There are other ODE algorithms. We have to balance computate speed and error for each one. A commonly used algorithm is Runge-Kutta (RK). Right now, we're going to focus on second-order Runge-Kutta (RK2).

## Second order Runge-Kutta

### Recall that a system of ordinary differential equations (ODE’s) can be written as
$$
\frac{du}{dt} = f (u, t)
$$
The vector of dependent variables is denoted $u$, and the independent variable is $t$. The right–hand side, $f(u,t)$, is a vector of functions of the dependent and independent variables. 

For the Euler method, we approximate the ODE’s at time $t = t_i \equiv i\Delta t$ by 
$$
\frac{du}{dt}\approx (u_{i+1} - u_i)/\Delta t = f(u_i,t_i)\Delta t,
$$
where $u_i \equiv u(t_i)$. Rearranging, we have the result
$$
u_{i+1} =u_i +f(u_i,t_i) \Delta t
$$
which allows us to solve for $u_{i+1}$ given $u_i$. 

The Taylor series expansion of $u(t_i + \Delta t)$ shows that the exact result is
$$
u_{i+1} =u_i +f(u_i,t_i)\Delta t+\mathcal{O}(\Delta t^2)
$$
where $\mathcal{O}(\Delta t^2)$ denotes terms of order  t2. Thus, each step of the Euler method introduces truncation errors proportional to $\Delta t^2$. Since the number of timesteps required to reach a finite time is proportional to $1/\Delta t$, the cumulative error in the Euler method is order $\Delta t$. We say that the Euler method is a _first order_ method, meaning that the errors are proportional to the first power of $\Delta t$.

Go back and look at your plots of error vs `dt` to confirm for yourself that this is the case.

The second order Runge–Kutta method (RK2) is defined by the following two–substep process:
$$
\begin{aligned}
u_h &= u_i +f(u_i,t_i)\Delta t/2 \\
u_{i+1} &= u_i +f(u_h,t_h)\Delta t
\end{aligned}
$$
where $t_h \equiv t_i +  \Delta t/2$. In the first substep, the Euler method is used to estimate the value of $u$ at the half timestep. That is, the value of $u$ at time $t_h \equiv t_i + \Delta t/2$. Here, this value is denoted $u_h$; sometimes it is denoted $u_{i+1/2}$. The second substep looks just like the Euler method, but instead of using $u_i$ and $t_i$ in the function $f$, we use the half–timestep values $u_h$ and $t_h$.

The second order Runge–Kutta method (RK2) is defined by the following two–substep process:
$$
\begin{aligned}
u_h &= u_i +f(u_i,t_i)\Delta t/2 \\
u_{i+1} &= u_i +f(u_h,t_h)\Delta t
\end{aligned}
$$
where $t_h \equiv t_i +  \Delta t/2$. In the first substep, the Euler method is used to estimate the value of $u$ at the half timestep. That is, the value of $u$ at time $t_h \equiv t_i + \Delta t/2$. Here, this value is denoted $u_h$; sometimes it is denoted $u_{i+1/2}$. The second substep looks just like the Euler method, but instead of using $u_i$ and $t_i$ in the function $f$, we use the half–timestep values $u_h$ and $t_h$.

In [ ]:
import numpy as np
import pylab as p

tau = 0.7        # mean lifetime, in gigayears
N0 = 100.0 # initial number of nuclei

tmin = 0.0
tmax = 4.0

nts=20

# Create an array to hold the times
t_array = np.linspace(tmin, tmax, nts, endpoint=False)  
dt = t_array[1] - t_array[0]  #  This is an easy way to define dt that is not error prone

# Empty array to hold the number of nuclei
N_array = np.zeros( len(t_array) )

# Empty array for Euler comparison
N_euler = np.zeros( len(t_array) )


# Do the simple Euler solution
# Set the initial condition
N_euler[0] = N0
for it in range(0, len(t_array)-1 ):
    N_euler[it+1] = N_euler[it] + dt * (-1./tau * N_euler[it])
    
# The above should have been familiar from Module 6.1

# Do the RK2 solution
N_array[0] = N0
for it in range(0, len(t_array)-1 ):
    t  = t_array[it]         # Don't necessarily need it but nice to have a simpler named variable
    th = t_array[it] + dt/2  # Note: we don't need this for this particular ODE
    
    N_h           = N_array[it] + dt # FILL THIS IN
    N_array[it+1] = N_array[it] + dt # FILL THIS IN
    
    
# Make a plot (note the label for the curve)
p.plot(t_array, N_euler, 'o', label="Euler Method with nts="+str(nts))
p.plot(t_array, N_array, 'o', label="RK2 Method with nts="+str(nts))
p.plot(t_array, N0*np.exp(-t_array/tau),'k:',label="Analytic result")

# Add a legend (based on the labels)
p.legend()

# Add a horizontal axis
p.axhline(0.0, color='k', linestyle='--')

# Show the plot
p.show()

A more usable differential equation solver
------------------------------
We’ve now used the Euler/RK2 methods to solve a few differential equations, and have probably either re-written the code or copied it previously. However, we can do better! We can re-write the code in a way that makes it usable for a variety of problems without changing the differential equation solver.  Consider the Euler solver we have been using:


In [ ]:
import numpy as np
import pylab as p

tau = 0.7        # mean lifetime
N0 = 100.0       # initial number of nuclei
tmin = 0.0       # start time
tmax = 4.0       # end time
nts=20           # number of points

# Create an array to hold the times and number of nuclei
t = np.linspace(tmin, tmax, nts + 1)  
N = np.zeros( len(t_array) )

dt = t[1] - t[0]  #  This is an easy way to define dt that is not error prone

# Do the simple Euler solution
N[0] = N0
for it in range(0, len(t)-1 ):
    N[it+1] = N[it] + dt* (-1./tau * N[it])

We can re-write the solver in its own function, and then pass in a separate function that knows how to compute the right hand side of the equation (the derivative):

In [ ]:
import numpy as np
import pylab as p

# This function calculates the RHS of the differential equation
# for the decay problem: just -(1/tau) * N
def decay_deriv(N,t):
    # The derivative in principle may depend on N and t -- in this
    # case, it only depends on N
    return -(1./tau) * N

# This function runs an Euler solver.  We have to tell it the initial
# value, the min/max time range, the number of time points,
# and how to compute the derivative (we pass in a function
# for this).
def Euler_solver(N_initial, tmin, tmax, nts, deriv):
    N = np.zeros(nts+1)
    t = np.linspace(tmin, tmax, nts+1)
    
    # Note: a way of defining dt that is less error-prone
    dt = t[1] - t[0]
    N[0] = N_initial
    for it in range(0,nts):
        N[it+1] = N[it] + dt * deriv(N[it], t[it])
    return t, N

tau = 0.7        # mean lifetime
N0 = 100.0       # initial number of nuclei
tmin = 0.0       # start time
tmax = 4.0       # end time
nts=20           # number of points

t,N_euler = Euler_solver(N0, tmin, tmax, nts, decay_deriv)

# I wonder what these things are?  Let's look at their shapes.
print ("Shapes:",np.shape(t), np.shape(N_euler))

p.plot(t,N_euler,'o-')
p.show()

## Digging in deeper 

Let's dissect the code above a little bit.  We've defined a few functions, we'll get to those in a minute.  Below those, we find the initial value (`N0`), the time axis parameters (`tmin, tmax, nts`) and the physics (`tau`). After that, we find this line:

    t,N_euler = Euler_solver(N0, tmin, tmax, nts, decay_deriv)
    
This calls the `Euler_solver` function with a bunch of parameters, and assigns the return values (__note there are two return values__) to the `t` and `N_euler` variables.  We won't _a priori_ know what they are, but we can check with a call to `shape()`.  It turns out these are length-21 arrays -- that kind of makes sense since that's what I expect from an Euler solver. (Note, I could have written it so that it takes a t array as input as well).  

As the last argument, we are passing in `decay_deriv`.  This is the name of the function that, given $N(t)$ and $t$, knows how to compute the derivative (i.e. it has the right hand side of the ODE in it).

Within `Euler_solver`, we find the usual, now-familiar setup for an Euler solver.  One line is somewhat different:

    N[it+1] = N[it] + dt * deriv(N[it], t[it])
    
In the past, we had the explicit form of the derivative for the nuclear decay problem at the end of that line.  Now, instead, we'll call yet another function which will compute it for us.  Just like the right hand side of a general ODE, it takes the function (here $N(t)$ and the independent variable $t$).

Since we passed `decay_deriv` in to the variable `deriv` of the `Euler_solver` function, that's what will be called to compute the derivative.

All of this structure is useful for a few things
1. Replacing the derivative function without having to rewrite the whole code
2. Understanding how to use better, pre-written differential equation solvers (next section)

## Exercises (On Your Own Time)

a) Well-commented code which uses your RK2 solver to compute the relative error at the last time point for at least four different resolutions (`nts`).

b) Make a plot of log10(error) vs log10(nts), and explain how this shows that the relative errors are proportional to $\Delta t^2$.

c) Compute the relative error in your RK2 code when the number of time steps is set to `nts=100`. Use your Euler code to find the (approximate) number of time steps required for Euler to give a result with comparable error.   

d) Write a similar function as `Euler_solver` called `RK2_solver` that takes the same arguments and returns the same values. It should use the RK2 algorithm to do the differential equation solving. (Hint: it should only require the addition of a line or two.)


For reference, SciPy
============================
There are quite a few ways of solving differential equations that are more complex (and more accurate) than both Euler and RK2 (for example, RK4). Fortunately for us, most of these have already been coded up, and we can simply call them rather than writing each and every one ourselves. Numpy doesn’t have any of these, but Scipy does. __Scipy__ is a large repository of scientific computing code, with many things already implemented, including differential equation solvers. Here, we will use odeint, which stands for Ordinary Differential Equation Integrator. You may find the documentation here: https://docs.scipy.org/doc/scipy/reference/generated/scipy.integrate.odeint.html. 

Have a look at the documentation, and the example at the bottom of the page. You’ll find that it takes about a million possible arguments, but only three are necessary: the function that computes the derivative, the initial value, and the time axis. We could thus define a function that looks like our Euler/RK2 solvers that calls the scipy integrator as follows:

    # We need to make sure that the code has access to the odeint function
    # so we import it
    from scipy.integrate import odeint
    def diffeq_solver_from_scipy(N_initial, tmin, tmax, nts, deriv):
        t = np.linspace(tmin, tmax, nts)
        
        # Note that the order of arguments matches the documentation
        N = odeint(deriv, N_initial, t)
        
        return t,N
        
Note the similarity to the code we wrote above in the `Euler_solver` function.  This was intentional -- this way, you should be able to simply exchange one function call for another.

_Scipy_ by default uses something called the Adams method for differential equation solving.


And, for reference, below is code that Caroline wrote during undergrad that shows basic implimentation of RK4. The original context of the homework problem has been lost (that's why it's super important to comment your code!!), but the point is that I wrote a function for the differential equation and then all I had to do was call `RK4`. If I had been really slick, I could have had the arguments that feed into the code include a function for a differential equation. But, the point is, the logic of RK4 is below if you have a function to define the associated differential equation.

In [ ]:
def f(r,t):             #differential equations
    
    x = r[0]
    y = r[1]
    vx = r[2]
    vy = r[3]
    r_sep = np.sqrt(x**2 + y**2)
    dvx = (-G*M*x)/(r_sep**2*np.sqrt(r_sep**2 +(L**2)/4))
    dvy = (-G*M*y)/(r_sep**2*np.sqrt(r_sep**2 +(L**2)/4))
    
    return np.array([vx,vy,dvx,dvy])

def RK4(r,t):           #fourth order Runge-Kutta
    a = 0
    b = 10
    N = 1000
    h = (b-a)/N
    k1 = h*f(r,t)
    k2 = h*f(r + .5 * k1,t +0.5*h)
    k3 = h * f(r + .5 * k2,t +0.5*h)
    k4 = h * f(r + k3, t+h)
    r = r + (1/6) * (k1 + 2 * k2 + 2 * k3 + k4)
    return r